About:

The main goal is to fine tune BERT model to classify phishing URLs.

In [1]:
!pip install datasets

In [2]:
!pip install transformers

In [3]:
!pip install evaluate

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments
import evaluate
import numpy as np


In [5]:
ds = load_dataset("shawhin/phishing-site-classification")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
model_path = 'google-bert/bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_path)

id2label, label2id = [{0: 'Not Safe', 1: 'Safe'}, {'Safe': 1, 'Not Safe': 0}]

model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [8]:
# freeze all the model layers, except a few ones
for name, param in model.base_model.named_parameters():
    if 'pooler' not in name:
        param.requires_grad = False

'''
After doing this we only have 4 trainable layers:
inside_bert_base_model:
    (pooler): BertPooler(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (activation): Tanh()
    )
outside_layers_we_added_using_AutoModelForSequenceClassification:
    (dropout): Dropout(p=0.1, inplace=False)
    (classifier): Linear(in_features=768, out_features=2, bias=True)
'''

'\nAfter doing this we only have 4 trainable layers:\ninside_bert_base_model:\n    (pooler): BertPooler(\n      (dense): Linear(in_features=768, out_features=768, bias=True)\n      (activation): Tanh()\n    )\noutside_layers_we_added_using_AutoModelForSequenceClassification:\n    (dropout): Dropout(p=0.1, inplace=False)\n    (classifier): Linear(in_features=768, out_features=2, bias=True)\n'

In [9]:
def tokenize_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True)

tokenized_ds = ds.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer)

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [10]:
tokenized_ds.shape

{'train': (2100, 5), 'validation': (450, 5), 'test': (450, 5)}

In [11]:
accuracy = evaluate.load('accuracy')
roc_auc_score = evaluate.load('roc_auc')

def compute_metrics(pred):
    predictions, labels = pred

    probabilities = np.exp(predictions)/np.exp(predictions).sum(-1, keepdims=True)

    positive_class_probs = probabilities[:, 1]

    auc = np.round(roc_auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'], 3)

    predicted_class = np.argmax(probabilities, axis=1)

    acc = np.round(accuracy.compute(predictions=predicted_class, references=labels)['accuracy'], 3)

    return {'Accuracy': acc, 'AUC': auc}

In [12]:
LEARNING_RATE = 2e-4
BATCH_SIZE = 8
EPOCHS = 10

training_args = TrainingArguments(
    output_dir='bert-fine-tuned-url-classification',
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    logging_strategy='epoch',
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_ds['train'],
    eval_dataset= tokenized_ds['test'],
    compute_metrics=compute_metrics
)

In [14]:
# Before Fine Tuning Predictions
initial_predictions = trainer.predict(tokenized_ds['validation'])
initial_metric = compute_metrics((initial_predictions.predictions, initial_predictions.label_ids))
print(initial_metric)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dhrroovv (dhrroovv-nitj) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'Accuracy': np.float64(0.5), 'AUC': np.float64(0.637)}


In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,Auc
1,0.500500,0.429073,0.004900,0.782000,0.907000
2,0.416800,0.360926,0.004900,0.831000,0.929000
3,0.368700,0.321725,0.004900,0.858000,0.936000
4,0.347100,0.351820,0.004900,0.853000,0.941000
5,0.346300,0.310670,0.004900,0.858000,0.946000
6,0.341300,0.291460,0.004900,0.867000,0.950000
7,0.335400,0.288500,0.004900,0.869000,0.949000
8,0.318800,0.287508,0.004900,0.862000,0.951000
9,0.315700,0.284823,0.004900,0.864000,0.951000
10,0.310300,0.293052,0.004900,0.869000,0.951000


TrainOutput(global_step=2630, training_loss=0.3600987945672677, metrics={'train_runtime': 794.2171, 'train_samples_per_second': 26.441, 'train_steps_per_second': 3.311, 'total_flos': 5525332162560000.0, 'train_loss': 0.3600987945672677, 'epoch': 10.0})

Evaluation Process

In [16]:
predictions = trainer.predict(tokenized_ds['validation'])

In [17]:
metric = compute_metrics((predictions.predictions, predictions.label_ids))  # prediction_logits, labels

In [18]:
print(metric)

{'Accuracy': np.float64(0.893), 'AUC': np.float64(0.946)}


In [19]:
import torch

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [21]:
teacher_model = model.to(device)

In [22]:
teacher_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [23]:
# loading the student model

from transformers import DistilBertForSequenceClassification, DistilBertConfig

my_config = DistilBertConfig(n_layers=4, n_heads=8)

student_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', config=my_config).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
print(sum(p.numel() for p in student_model.parameters())) # total number of parameters in the student_model

52779266


In [25]:
tokenized_ds.shape

{'train': (2100, 5), 'validation': (450, 5), 'test': (450, 5)}

In [26]:
tokenized_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [27]:
print(tokenized_ds.shape)

{'train': (2100, 5), 'validation': (450, 5), 'test': (450, 5)}


In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [29]:
from torch.utils.data import DataLoader

def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_masks = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            output = model(input_ids, attention_mask=attention_masks)
            logits = output.logits

            predictions = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(predictions)
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')

    return accuracy, precision, recall, f1

In [30]:
def distillation_loss(student_logits, teacher_logits, actual_labels, temperature, alpha):
    teacher_soft = F.softmax(teacher_logits/temperature, dim=1)  # smoothening the teacher output
    student_soft = F.log_softmax(student_logits/temperature, dim=1)  # smoothening the student output

    distill_loss = F.kl_div(student_soft, teacher_soft, reduction='batchmean') * (temperature ** 2)

    cel = nn.CrossEntropyLoss()
    hard_loss = cel(student_logits, actual_labels)

    loss = alpha * distill_loss + (1 - alpha) * hard_loss

    return loss

In [31]:
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
EPOCHS = 5
TEMPERATURE = 2.0
ALPHA = 0.5

optimizer = optim.Adam(student_model.parameters(), lr=LEARNING_RATE)

train_dataloader = DataLoader(tokenized_ds['train'], batch_size=BATCH_SIZE)
test_dataloader = DataLoader(tokenized_ds['test'], batch_size=BATCH_SIZE)

In [32]:
student_model.train()

for epoch in range(EPOCHS):
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)

        with torch.no_grad():  # disable the gradient calc for teacher model -> bcz obv we are not training the teacher
            teacher_output = teacher_model(input_ids, attention_mask=attention_mask)
            teacher_logits = teacher_output.logits

        student_output = student_model(input_ids, attention_mask=attention_mask)
        student_logits = student_output.logits

        loss = distillation_loss(student_logits, teacher_logits, labels, TEMPERATURE, ALPHA)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch: {epoch+1}    Loss: {loss.item()}')

Epoch: 1    Loss: 0.160567045211792
Epoch: 2    Loss: 0.12642018496990204
Epoch: 3    Loss: 0.06368622183799744
Epoch: 4    Loss: 0.04971132427453995
Epoch: 5    Loss: 0.09538456052541733


In [33]:
validation_dataloader = DataLoader(tokenized_ds['validation'], batch_size=8)

# put the models to evaluation mode
teacher_model.eval()
student_model.eval()

# teacher
t_a, t_p, t_r, t_f = evaluate_model(teacher_model, validation_dataloader)

print(f"Teacher (validation) - Accuracy: {t_a}, Precision: {t_p}, Recall: {t_r}, F1 Score: {t_f}")


# student
s_a, s_p, s_r, s_f = evaluate_model(student_model, validation_dataloader)

print(f"Student (validation) - Accuracy: {s_a}, Precision: {s_p}, Recall: {s_r}, F1 Score: {s_f}")

Teacher (validation) - Accuracy: 0.8933333333333333, Precision: 0.9116279069767442, Recall: 0.8711111111111111, F1 Score: 0.8909090909090909
Student (validation) - Accuracy: 0.9133333333333333, Precision: 0.965, Recall: 0.8577777777777778, F1 Score: 0.908235294117647


**Remarks**:

Student Model is performing better than Teacher Model

In [34]:
from huggingface_hub import notebook_login
notebook_login()


In [35]:
student_model.push_to_hub('dhrroovv/distilbert-url-classifier')

model.safetensors:   0%|          | 0.00/211M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dhrroovv/distilbert-url-classifier/commit/e0ed6f6e9d8a56e47f69c717e2fd1471b69677f4', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='e0ed6f6e9d8a56e47f69c717e2fd1471b69677f4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dhrroovv/distilbert-url-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='dhrroovv/distilbert-url-classifier'), pr_revision=None, pr_num=None)

In [36]:
tokenizer.push_to_hub('dhrroovv/distilbert-url-classifier')

CommitInfo(commit_url='https://huggingface.co/dhrroovv/distilbert-url-classifier/commit/ee60c0bf709b3e8707fc3722504269ffface247f', commit_message='Upload tokenizer', commit_description='', oid='ee60c0bf709b3e8707fc3722504269ffface247f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dhrroovv/distilbert-url-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='dhrroovv/distilbert-url-classifier'), pr_revision=None, pr_num=None)

In [37]:
my_config.push_to_hub('dhrroovv/distilbert-url-classifier')

CommitInfo(commit_url='https://huggingface.co/dhrroovv/distilbert-url-classifier/commit/cfbe29190f7c978db2832abe644371e9f7384e81', commit_message='Upload config', commit_description='', oid='cfbe29190f7c978db2832abe644371e9f7384e81', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dhrroovv/distilbert-url-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='dhrroovv/distilbert-url-classifier'), pr_revision=None, pr_num=None)

Quantization

In [38]:
!pip install -U bitsandbytes


In [39]:
from transformers import BitsAndBytesConfig

# nf4 -> 4-bit normal weight
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype = torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

model_nf4 = AutoModelForSequenceClassification.from_pretrained('dhrroovv/distilbert-url-classifier',
                                                device_map=device,
                                                quantization_config=nf4_config)

model.safetensors:   0%|          | 0.00/211M [00:00<?, ?B/s]

In [40]:
final_model_a, final_model_p, final_model_r, final_model_f = evaluate_model(model_nf4, validation_dataloader)
print(f"Final Model (validation) - Accuracy: {final_model_a}, Precision: {final_model_p}, Recall: {final_model_r}, F1 Score: {final_model_f}")

Final Model (validation) - Accuracy: 0.9088888888888889, Precision: 0.9646464646464646, Recall: 0.8488888888888889, F1 Score: 0.9030732860520094


In [ ]:
# sharing the final model on huggingface

model_nf4.push_to_hub('dhrroovv/distilbert-url-classifier')
tokenizer.push_to_hub('dhrroovv/distilbert-url-classifier')